In [1]:
from unis.models import *
from unis.runtime import Runtime
from unis.rest import UnisReferenceError
from unis.models.lists import UnisCollection
import itertools

In [2]:
def commit(target):
    collections = ["ports", "nodes", "links", "paths", "networks", 
                   "domains", "topologies", "exnodes", "extents"]
    unordered = filter(lambda x: x not in collections, target.collections)
    collections.extend(unordered)
    for collection in collections:
        for item in getattr(target, collection):
            item.commit()


In [3]:
def node(rt, name):
    node = Node({"name": name})
    rt.insert(node)
    return node

def port(rt, node):
    port = Port({"name": ":".join([node.name, "port1"])})
    rt.insert(port)
    node.ports = [port]
    return port

def link(rt, mapping):
    left = next(rt.ports.where({"name": ":".join([mapping[0], "port1"])}))
    right = next(rt.ports.where({"name": ":".join([mapping[1], "port1"])}))
    link = Link({"name": "--".join([left.name, right.name]), 
                 "directed": False, 
                 "endpoints": [left, right]})
    rt.insert(link)
    return link
    
def domain(rt, name):
    nodes = list(rt.nodes.where(lambda x: x.name == name))
    ports = list(rt.ports.where(lambda x: x.name.startswith(name)))
    links = list(rt.links.where(lambda x: x.endpoints[0].name.startswith(name)))
    domain = Domain({"name": name, "nodes": nodes, "ports": ports, "links":links})
    rt.insert(domain)
    return domain

def topology(rt, name, ports, nodes, links, domains):
    topo = Topology({"name": name, "ports": ports, 
                     "nodes": nodes, "links": links, 
                     "domains": domains})
    rt.insert(topo)

In [4]:
def osiris(url):
    with Runtime(url) as rt:
        domain_names = ["IU", "WSU", "MSU", "CHIC", "SALT", 
                        "SC16", "IU-Crest", "UMich", "Cloudlab"]
        link_map = [("IU", "CHIC"), ("UMich", "CHIC"), ("WSU", "CHIC"), 
                    ("MSU", "CHIC"), ("CHIC", "SALT"), ("Cloudlab", "SALT"), 
                    ("SC16", "SALT"), ("SC16", "UMich"), ("SC16", "IU-Crest")]
        nodes = [node(rt, d) for d in domain_names]
        ports = [port(rt, n) for n in nodes]
        links = [link(rt, l) for l in link_map]
        domains = [domain(rt, d) for d in domain_names]
        topology(rt, "OSiRIS", nodes, ports, links, domains)
        commit(rt)

In [5]:
def ring_spur(url, ring, spurs):
    def name(link, spur): return "Node-{0}-{1}".format(link, spur)
    
    node_names = [name(i, j)
                  for j in range(spurs)
                  for i in range(ring)]
    ring_links = [(name(i,0), name((i+1)%ring, 0))
                  for i in range(ring)]
    spur_links = [(name(i,j), name(i, 0))
                  for j in range(1, spurs)
                  for i in range(ring)]
    
    link_map = list(itertools.chain(ring_links, spur_links))

    with Runtime(url) as rt:
        nodes = [node(rt, d) for d in node_names]
        ports = [port(rt, n) for n in nodes]
        links = [link(rt, l) for l in link_map]
        domain = Domain({"name": "ring-domain", "nodes": nodes, "ports": ports, "links": links})
        rt.insert(domain)
        topology(rt, "ring", nodes, ports, [], [domain])
        commit(rt)
        

In [6]:
target = "http://192.168.100.100:8888"
osiris(target)
#ring_spur(target, 10, 3)

ValueError: Resource type not found in ObjectLayer

In [15]:
p = Port()

In [7]:
rt = Runtime("http://192.168.100.200:8888")

In [12]:
switch_node = Node({"name": "test_switch"})
port_object = Port({"name": "test_port"})
switch_node.ports.append(port_object)
rt.insert(switch_node)

ValueError: Resource type not found in ObjectLayer